## Note
I've always struggled with "setting up" environments to play in. SQL and SQL databases have always confused me on how to properly get them up and running. Once they're up though, then it's relatively easy. I never learned a simple/easy way of setting up an SQL playground, so I hope this will suffice! I will fully admit that my SQL skills are probably mediocre? It's not a skill I got to practice regularly, so I'm sure once I am in a situation where I use it daily, I'll improve quickly. ^^

In [1]:
#import tools
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [2]:
#files
merch2021 = 'lafc_sql_test_2021merch.xlsx'
merch2022 = 'lafc_sql_test_2022merch.xlsx'
events    = 'lafc_sql_test_events.xlsx'
manifest  = 'lafc_sql_test_manifest.xlsx'
tickets   = 'lafc_sql_test_tickets.xlsx'

output = 'output.xlsx'
engine = create_engine('sqlite://', echo=False)

In [3]:
#read in all the sheets
merch2021_df = pd.read_excel(merch2021, sheet_name='Sheet1')
merch2022_df = pd.read_excel(merch2022, sheet_name='in')
events_df    = pd.read_excel(events, sheet_name='in')
manifest_df  = pd.read_excel(manifest, sheet_name='in')
tickets_df   = pd.read_excel(tickets, sheet_name='in')

tickets_df.to_sql('tickets', engine, if_exists='replace', index=False)
manifest_df.to_sql('manifest', engine, if_exists='replace', index=False)
events_df.to_sql('events', engine, if_exists='replace', index=False)
merch2021_df.to_sql('merch2021', engine, if_exists='replace', index=False)
merch2022_df.to_sql('merch2022', engine, if_exists='replace', index=False)

3784

In [4]:
#test run, just to make sure it works :)
results = engine.execute("SELECT * FROM tickets")
final = pd.DataFrame(results)
final.head()

,event_name,section_name,row_name,num_seats,ticket_status,acct_id,price_code,comp_name,purchase_price,ticket_type,price_code_desc,event_id,plan_event_name,tran_type,section_id,row_id,add_datetime,seat_num
0,LAF10903,SUPP3,GA8,1,A,22464,TR3,Not Comp,14.000,Renewal 3,None,430,21FS13,Plans,1459,9,00:38:45.500000,12
1,LAF10724,122,J,1,A,15575,QER3,Not Comp,26.165,Renewal 3,None,426,21FS13,Plans,1289,9,00:20:06.500000,18
2,LAF10724,122,J,1,A,15575,QER3,Not Comp,26.165,Renewal 3,None,426,21FS13,Plans,1289,9,00:20:06.500000,20
3,LAF10804,SUPP2,GA4,1,A,18168,TR3,Not Comp,14.000,Renewal 3,None,428,21FS13,Plans,1458,5,00:29:59.300000,46
4,LAF10929,120,N,1,A,109462,Q3N,Not Comp,22.200,New Sale,120 M-R 124 L-R,432,None,event,1287,13,00:44:16.900000,16


# Query 1 - Section Tickets for Every Account ID

In [5]:
#QUERY 1 - SECTION TICKETS FOR EVERY ACCOUNT ID
query = """
SELECT acct_id, section_name, plan_event_name
FROM tickets
WHERE plan_event_name LIKE '22FS%'
GROUP BY acct_id;
"""

results = engine.execute(query)
df = pd.DataFrame(results)
df

,acct_id,section_name,plan_event_name
0,10001,FLDCLB,22FS
1,10003,FNDCLE,22FS
2,10007,FLDCLA,22FS
3,10009,FNDCLB,22FS
4,10011,FLDCLD,22FS
...,...,...,...
6518,10732818,FLDCLD,22FS14
6519,10968495,101,22FS14
6520,10990192,211,22FS13
6521,10991687,FIGCLB,22FS13


# Query 2 - Total Amount Spent for Every Account ID

In [6]:
#QUERY 2 - TOTAL AMOUNT SPENT FOR EVERY ACCOUNT ID
query = """
SELECT acct_id, SUM(purchase_price) TotalSpent
FROM (
    SELECT acct_id, purchase_price FROM tickets UNION ALL
    SELECT acct_id, line_total FROM merch2021 UNION ALL
    SELECT acct_id, line_total FROM merch2022
    ) x
GROUP BY acct_id;
"""

results = engine.execute(query)
final = pd.DataFrame(results)
final

,acct_id,TotalSpent
0,-2,568.5000
1,503,907.0000
2,10001,5197.9450
3,10003,29295.1000
4,10007,27392.9200
...,...,...
17955,11361499,53.9798
17956,11363022,21.9899
17957,11363376,4.9900
17958,11364125,61.4799


# Query 3 - Earliest Purchase Date for Every Account ID

In [7]:
#QUERY 3 - EARLIEST PURCHASE DATE FOR EVERY ACCOUNT ID
query = """
SELECT acct_id, MIN(first_order_date) FirstOrder
FROM (
    SELECT acct_id, first_order_date FROM merch2021 UNION ALL
    SELECT acct_id, first_order_date FROM merch2022
    ) x
GROUP BY acct_id;
"""

results = engine.execute(query)
final = pd.DataFrame(results)
final

,acct_id,FirstOrder
0,10009,2017-06-25 00:00:00.000000
1,10018,2017-07-28 00:00:00.000000
2,10027,2018-10-25 00:00:00.000000
3,10035,2017-08-29 00:00:00.000000
4,10036,2018-05-18 00:00:00.000000
...,...,...
6695,11361399,2021-11-08 00:00:00.000000
6696,11361499,2022-03-11 00:00:00.000000
6697,11363022,2021-08-20 00:00:00.000000
6698,11363376,2021-05-15 00:00:00.000000


# Query 4 - Sell Through Rate by Section for Final Game
This last one took me some time, so I kept the other queries I used to test before I made the final query.

In [8]:
# ACTIVE TICKETS FOR FINAL GAME BY SECTION --- DON'T ERASE
query = """
    SELECT e.event_name Event, t.section_name Section, COUNT(*) Tickets_Sold
    FROM events e
    INNER JOIN tickets t ON t.event_name = e.event_name
    WHERE e.game_number = 17 AND e.event_date >= '2022-10-09' AND t.ticket_status = 'A'
    GROUP BY t.section_name
"""

results = engine.execute(query)
q4a = pd.DataFrame(results)
q4a

,Event,Section,Tickets_Sold
0,LAF21009,101,262
1,LAF21009,102WC,10
2,LAF21009,104WC,10
3,LAF21009,105WC,15
4,LAF21009,107,210
...,...,...,...
192,LAF21009,SUPP2,479
193,LAF21009,SUPP3,498
194,LAF21009,SUPP4,499
195,LAF21009,SUPP5,410


In [9]:
# MANIFEST SEATS BY SECTION --- DON'T ERASE
query = """
SELECT section_name Section, SUM(num_seats) Total_Seats
FROM manifest
GROUP BY section_name
"""

results = engine.execute(query)
q4b = pd.DataFrame(results)
q4b

,Section,Total_Seats
0,101,275
1,102WC,14
2,104WC,12
3,105WC,18
4,107,210
...,...,...
208,SUPP2,500
209,SUPP3,500
210,SUPP4,500
211,SUPP5,500


In [10]:
#QUERY 4 - SELL_THRU_RATE BY SECTION FOR THE LAST REGULAR SEASON GAME
query = """
WITH manifest_seats AS (
    SELECT section_name Section, 
           SUM(num_seats) Total_Seats
    FROM manifest
    GROUP BY section_name
),

    active_tickets_by_section AS (
    SELECT e.event_name Event, t.section_name Section, COUNT(*) Tickets_Sold
    FROM events e
    INNER JOIN tickets t ON t.event_name = e.event_name
    WHERE e.game_number = 17 AND e.event_date >= '2022-10-09' AND t.ticket_status = 'A'
    GROUP BY t.section_name
)

SELECT a.Event,
       a.Section,
       a.Tickets_Sold,
       m.Total_Seats,
       ROUND(a.Tickets_Sold * 100.0 / m.Total_Seats,1) Sell_Thru_Rate
FROM active_tickets_by_section a
INNER JOIN manifest_seats m ON m.Section = a.Section
GROUP BY a.Section;
"""

results = engine.execute(query)
q4 = pd.DataFrame(results)
q4

,Event,Section,Tickets_Sold,Total_Seats,Sell_Thru_Rate
0,LAF21009,101,262,275,95.3
1,LAF21009,102WC,10,14,71.4
2,LAF21009,104WC,10,12,83.3
3,LAF21009,105WC,15,18,83.3
4,LAF21009,107,210,210,100.0
...,...,...,...,...,...
192,LAF21009,SUPP2,479,500,95.8
193,LAF21009,SUPP3,498,500,99.6
194,LAF21009,SUPP4,499,500,99.8
195,LAF21009,SUPP5,410,500,82.0
